Работа с .npy форматом, загрузка 

In [ ]:
!python3 test.py

In [ ]:
from dataloader import SVGDataset
import torchvision.transforms as T
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch
import math

def set_range(X):
    return 1. - X

SetRange = T.Lambda(set_range)  # convert [0, 1] -> [0, 1]
transform = T.Compose([SetRange])
dataset = SVGDataset("root", 64, 'eng', 52, 51, 12, transform, 'train')

class_data = dataset[0]['seq_len'] # чтобы обратиться по ключу надо взять 0 индекс

dataloader = data.DataLoader(dataset, 1, shuffle= 'train', num_workers=0) # торчовская функция 

for idx, data in enumerate(dataloader):
    for key, value in data.items():
        print(f"Key: {key}, Shape: {value.shape}")
                

In [ ]:
import torch
#print(data['sequence'])
#print(data['sequence'].shape)
input_sequence = data['sequence']
input_pts_aux = data['pts_aux']
def numericalize(cmd, n=128):
    """NOTE: shall only be called after normalization"""
    # assert np.max(cmd.origin) <= 1.0 and np.min(cmd.origin) >= -1.0 
    cmd = (cmd / 30 * n).round().clip(min=0, max=n-1).int()
    return cmd
arg_quant = numericalize(input_sequence[:, :, :, 4:])
cmd_cls = torch.argmax(input_sequence[:, :, :, :4], dim=-1).unsqueeze(-1)
input_sequence = torch.cat([cmd_cls, arg_quant], dim=-1)

print(input_pts_aux[0][0][0])

In [ ]:
input_image = data['rendered']
ref_cls = torch.randint(0, 52, (1, 4))
trg_cls = torch.randint(0, 52, (1, 1))

"""
selected_cls = trg_cls
selected_cls_ = selected_cls.unsqueeze(2)
selected_cls_ = selected_cls_.unsqueeze(3)
print(selected_cls_)
selected_cls_ = selected_cls_.expand(1, 4, 64, 64)
selected_img = torch.gather(input_image, 1, selected_cls_)
print(selected_img[0][0][0])
"""

def select_imgs(images_of_onefont, selected_cls):
    # given selected char classes, return selected imgs
    # images_of_onefont: [bs, 52, opts.img_size, opts.img_size]
    # selected_cls: [bs, nshot]
    nums = selected_cls.size(1)
    selected_cls_ = selected_cls.unsqueeze(2)
    selected_cls_ = selected_cls_.unsqueeze(3)
    selected_cls_ = selected_cls_.expand(images_of_onefont.size(0), nums, 64, 64)         
    selected_img = torch.gather(images_of_onefont, 1, selected_cls_)
    return selected_img

ref_img = select_imgs(input_image, ref_cls)
trg_img = select_imgs(input_image, trg_cls)


In [ ]:

"""
def select_seqs(seqs_of_onefont, selected_cls, opts, seq_dim):

    nums = selected_cls.size(1)
    selected_cls_ = selected_cls.unsqueeze(2)
    selected_cls_ = selected_cls_.unsqueeze(3)
    selected_cls_ = selected_cls_.expand(seqs_of_onefont.size(0), nums, opts.max_seq_len, seq_dim) 
    selected_seqs = torch.gather(seqs_of_onefont, 1, selected_cls_)
    return selected_seqs
"""

def select_seqs(seqs_of_onefont, selected_cls, n):
    nums = selected_cls.size(1)
    selected_cls_ = selected_cls.unsqueeze(2)
    selected_cls_ = selected_cls_.unsqueeze(3)
    selected_cls_ = selected_cls_.expand(seqs_of_onefont.size(0), nums, 51, n) 
    selected_seqs = torch.gather(seqs_of_onefont, 1, selected_cls_)
    return selected_seqs

print(select_seqs(input_sequence, trg_cls, 9))
print(select_seqs(input_sequence, trg_cls, 9).shape)



In [ ]:
trg_seq = select_seqs(input_sequence, trg_cls, 9) 
trg_seq = trg_seq.squeeze(1)

ref_seq = select_seqs(input_sequence, ref_cls, 9) 
print(trg_seq)
print(ref_seq)

In [ ]:
trg_pts_aux = select_seqs(input_pts_aux, trg_cls, 6)
trg_pts_aux = trg_pts_aux.squeeze(1)
print(trg_pts_aux)
print(trg_pts_aux.shape)

In [ ]:
trg_chat_onehot = F.one_hot(trg_cls, num_classes = 52).squeeze(dim = 1)

In [ ]:
trg_seq = select_seqs(input_sequence, trg_cls, 9) 
trg_seq = trg_seq.squeeze(1)
trg_seq_gt = trg_seq.clone().detach()
trg_seq_gt = torch.cat((trg_seq_gt[:, :, :1], trg_seq_gt[:, :, 3:]), -1)

trg_seq = trg_seq.transpose(0, 1)

trg_seq_shifted = F.pad(trg_seq, (0, 0, 0, 0, 1, 0))[:-1, :, :]
print(trg_seq_shifted)
print(trg_seq_shifted.shape)

In [ ]:
print(trg_seq)

In [ ]:
input_seqlen = data['seq_len'] 
input_seqlen = input_seqlen + 1


def select_seqlens(seqlens_of_onefont, selected_cls):

    nums = selected_cls.size(1)
    selected_cls_ = selected_cls.unsqueeze(2)
    selected_cls_ = selected_cls_.expand(seqlens_of_onefont.size(0), nums, 1)     # 64, nums, 1    
    selected_seqlens = torch.gather(seqlens_of_onefont, 1, selected_cls_)
    return selected_seqlens

ref_seqlen = select_seqlens(input_seqlen, ref_cls)
ref_seqlen_cat = ref_seqlen.view(ref_seqlen.size(0) * ref_seqlen.size(1), ref_seqlen.size(2))
print(ref_seqlen_cat)

In [ ]:
ref_img = select_imgs(input_image, ref_cls)
trg_img

In [ ]:
"""
print(ref_pad_mask)
"""
#ref_img # [bs, nshot, 64, 64]
#trg_img # [bs, 1, 64, 64]


# dim = 9, 1 command + 8 на точки
#ref_seq # [bs, nshot, max_seq_len, dim]
#ref_seq_cat # [max_seq_len, bs * nshot, dim]

#trg_seq # [max_seq_len, bs, dim]
#trg_seq_gt # [bs, max_seq_len, 7] dim - 2 (начальная точка = конечной предыдущей)
#trg_seq_shifted # [max_seq_len, bs, dim]
#trg_pts_aux # [bs, max_seq_len, 6] вспомогательные точки


#trg_chat_onehot # [bs, onehot]
#trg_cls # [bs, number_char] номера для trg
#trg_seqlen # вектор длин trg


In [ ]:
print(trg_seq)
print(trg_seq_gt)


Енкодер PNG

In [ ]:
ref_img

img_size = 64 
input_nc = 4
ngf = 16

n_downsampling = int(math.log(img_size, 2)) #  количество слоев свертки 
ks_list = [5] * (n_downsampling - n_downsampling // 3) + [3] * (n_downsampling // 3) # размер ядра для каждого слоя
stride_list = [2] * n_downsampling # страйд для каждого слоя 

chn_mult = []
for i in range(n_downsampling):
    chn_mult.append(2 ** (i + 1)) # число каналов на каждом слое

encoder = [nn.Conv2d(input_nc, ngf, kernel_size=7, padding=7 // 2, bias=True, padding_mode='replicate'),
            nn.LayerNorm([ngf, 2 ** n_downsampling, 2 ** n_downsampling]),
            nn.ReLU(True)]

for i in range(n_downsampling):  # add downsampling layers
    if i == 0:
        chn_prev = ngf
    else:
        chn_prev = ngf * chn_mult[i - 1]
    chn_next = ngf * chn_mult[i]

    encoder += [nn.Conv2d(chn_prev, chn_next, kernel_size=ks_list[i], stride=stride_list[i], padding=ks_list[i] // 2, padding_mode='replicate'),
                nn.LayerNorm([chn_next, 2 ** (n_downsampling - 1 - i), 2 ** (n_downsampling - 1 - i)]),
                nn.ReLU(True)]

encode = nn.Sequential(*encoder)
flatten = nn.Flatten(encode) # превращение в одномерный 


print(flatten)

Encoder SEQ